# Part 1 - Extracting and Saving Data from Yelp API

## Obective

- For this CodeAlong, we will be working with the Yelp API. 
- You will use the the Yelp API to search your home town for a cuisine type of your choice.
- Next class, we will then use Plotly Express to create a map with the Mapbox API to visualize the results.
    
    

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- Efficient API Calls Lesson Link: https://login.codingdojo.com/m/376/12529/88078

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
!pip install yelpapi

## 1. Registering for Required APIs


- Yelp: https://www.yelp.com/developers/documentation/v3/get_started


> Check the official API documentation to know what arguments we can search for: https://www.yelp.com/developers/documentation/v3/business_search

### Load Credentials and Create Yelp API Object

In [3]:
# Load API Credentials
import json
with open('/Users/joshl/.secret/yelp_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [4]:
# Instantiate YelpAPI Variable
# Create an instance with your key
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

### Define Search Terms and File Paths

In [6]:
# set our API call parameters and filename before the first call
# set our API call parameters 
LOCATION = 'Reno, NV'
TERM = 'Sushi'

In [7]:
## Specify fodler for saving data
FOLDER = 'Data/'

os.makedirs(FOLDER,exist_ok=True)

# Specifying JSON_FILE filename (can include a folder)
JSON_FILE = FOLDER+f"{LOCATION.split(',')[0]}-{TERM}.json"

In [8]:
JSON_FILE

'Data/Reno-Sushi.json'

### Check if Json File exists and Create it if it doesn't

In [9]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")


[i] Data/Reno-Sushi.json not found. Saving empty list to file.


### Load JSON FIle and account for previous results

In [10]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


### Make the first API call to get the first page of data

- We will use this first result to check:
    - how many total results there are?
    - Where is the actual data we want to save?
    - how many results do we get at a time?


In [14]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(term=TERM,location = LOCATION)

In [17]:
## How many results total?
results['total']

262

In [18]:
results['region']

{'center': {'longitude': -119.85260009765625, 'latitude': 39.549920383112536}}

In [20]:
results['businesses']

[{'id': 'pms70Lg_z7c2y5ojukohBQ',
  'alias': 'miyabi-japanese-cuisine-reno',
  'name': 'MIYABI Japanese Cuisine',
  'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/hE5VjDCn8xsOEgyEqxXzIA/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/miyabi-japanese-cuisine-reno?adjust_creative=qifpS3Fk20rXEHrtWOhzJA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=qifpS3Fk20rXEHrtWOhzJA',
  'review_count': 144,
  'categories': [{'alias': 'sushi', 'title': 'Sushi Bars'},
   {'alias': 'salad', 'title': 'Salad'},
   {'alias': 'japanese', 'title': 'Japanese'}],
  'rating': 4.5,
  'coordinates': {'latitude': 39.4035814750889,
   'longitude': -119.74875962824723},
  'transactions': [],
  'price': '$$',
  'location': {'address1': '13967 S Virginia St',
   'address2': 'Ste 910',
   'address3': None,
   'city': 'Reno',
   'zip_code': '89511',
   'country': 'US',
   'state': 'NV',
   'display_address': ['13967 S Virginia St', 'Ste 910', 'Reno, NV 89511']},
  'phone':

- Where is the actual data we want to save?

In [21]:
pd.DataFrame(results['businesses'])

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,pms70Lg_z7c2y5ojukohBQ,miyabi-japanese-cuisine-reno,MIYABI Japanese Cuisine,https://s3-media1.fl.yelpcdn.com/bphoto/hE5VjD...,False,https://www.yelp.com/biz/miyabi-japanese-cuisi...,144,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 39.4035814750889, 'longitude': -1...",[],$$,"{'address1': '13967 S Virginia St', 'address2'...",+17754093520,(775) 409-3520,18553.049571
1,W9y4JkMqy2dFNwYc3Y5cKQ,hinoki-sushi-reno,Hinoki Sushi,https://s3-media3.fl.yelpcdn.com/bphoto/vwh7Ea...,False,https://www.yelp.com/biz/hinoki-sushi-reno?adj...,843,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.0,"{'latitude': 39.473410597855896, 'longitude': ...",[delivery],$$,"{'address1': '5270 Longley Ln', 'address2': ''...",+17754094994,(775) 409-4994,11412.584176
2,3FPi2yKCIh0Hh3iBg8faYA,tokyo-sushi-reno-2,Tokyo Sushi,https://s3-media4.fl.yelpcdn.com/bphoto/WVhsaZ...,False,https://www.yelp.com/biz/tokyo-sushi-reno-2?ad...,978,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.0,"{'latitude': 39.50491, 'longitude': -119.80335}",[],$$,"{'address1': '1999 S Virginia St', 'address2':...",+17758258828,(775) 825-8828,6560.327423
3,gExn3d0VMMJWlSICsGHGfg,ohana-sushi-sparks-3,Ohana Sushi,https://s3-media2.fl.yelpcdn.com/bphoto/nMfIDn...,False,https://www.yelp.com/biz/ohana-sushi-sparks-3?...,495,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 39.52066, 'longitude': -119.740751}",[],$$,"{'address1': '1560 S Stanford Way', 'address2'...",+17754539722,(775) 453-9722,10134.379359
4,mc-U-3T6UMN9x7SVm9Yn3A,hinoki-o-reno-2,Hinoki O,https://s3-media4.fl.yelpcdn.com/bphoto/ZsNOKx...,False,https://www.yelp.com/biz/hinoki-o-reno-2?adjus...,26,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",5.0,"{'latitude': 39.46823189149659, 'longitude': -...",[],NaN,"{'address1': '7500 Rancharrah Pkwy', 'address2...",+17754995818,(775) 499-5818,10421.916314
5,n1ILbrSXiERCMR6w2cBPJA,o-a-sushi-reno,O’A Sushi,https://s3-media1.fl.yelpcdn.com/bphoto/sBFacC...,False,https://www.yelp.com/biz/o-a-sushi-reno?adjust...,107,"[{'alias': 'sushi', 'title': 'Sushi Bars'}]",4.5,"{'latitude': 39.4841, 'longitude': -119.74679}","[delivery, pickup]",$$,"{'address1': '3326 S McCarran Blvd', 'address2...",+17754486363,(775) 448-6363,11668.776496
6,vHr8qhM4CXYB3Ol_9yS6QQ,sushi-pier-reno,Sushi Pier,https://s3-media2.fl.yelpcdn.com/bphoto/LEgsB4...,False,https://www.yelp.com/biz/sushi-pier-reno?adjus...,1082,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.0,"{'latitude': 39.5044132846797, 'longitude': -1...",[delivery],$$$,"{'address1': '1290 E Plumb Ln', 'address2': 'S...",+17758256776,(775) 825-6776,7781.577324
7,q7mKAD93-EeSpeOa9Hu4BQ,sakana-sushi-bar-and-japanese-restaurant-reno,Sakana Sushi Bar & Japanese Restaurant,https://s3-media2.fl.yelpcdn.com/bphoto/Apj9VA...,False,https://www.yelp.com/biz/sakana-sushi-bar-and-...,334,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.0,"{'latitude': 39.531853, 'longitude': -119.91288}",[delivery],$$,"{'address1': '7655 Town Square Ln', 'address2'...",+17753761454,(775) 376-1454,5545.696235
8,36P_UsZOcTUSJ6Qok8HS3A,sumi-sushi-carson-city,Sumi Sushi,https://s3-media3.fl.yelpcdn.com/bphoto/tKY1eo...,False,https://www.yelp.com/biz/sumi-sushi-carson-cit...,12,"[{'alias': 'sushi', 'title': 'Sushi Bars'}]",4.0,"{'latitude': 39.165741, 'longitude': -119.766713}","[delivery, pickup]",NaN,"{'address1': '319 N Carson St', 'address2': No...",+17758832372,(775) 883-2372,43352.952471
9,plobBsTtVUODb353xxFT_g,sushi-rose-reno,Sushi Rose,https://s3-media1.fl.yelpcdn.com/bphoto/oLjAA1...,False,https://www.yelp.com/biz/sushi-rose-reno?adjus...,674,"[{'alias': 'sushi', 'title': 'Sushi Bars'}]",3.5,"{'latitude': 39.5347847599323, 'longitude': -1...","[delivery, restaurant_reservation]",$$,"{'address1': '2005 Sierra Highlands Dr', 'addr...",+17757471888,(775) 747-1888,2217.617394


In [22]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

- Calculate how many pages of results needed to cover the total_results

In [23]:
# Use math.ceil to round up for the total number of pages of results.
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

14

In [25]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    try:
        
        ## Read in results in progress file and check the length
        with open(JSON_FILE, 'r') as f:
            previous_results = json.load(f)
        ## save number of results for to use as offset
        n_results = len(previous_results)
        ## use n_results as the OFFSET 
        results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
        ## append new results and save to file
        ## extend allows multiple records to be saved
        previous_results.extend(results['businesses'])
    
        with open(JSON_FILE,'w') as f:
            json.dump(previous_results,f)
    
        # add a 200ms pause
        time.sleep(.2)
        
    except Exception as e:
        print('[!] ERROR: ',e)
        


  0%|          | 0/14 [00:00<?, ?it/s]

[!] ERROR:  [Errno 13] Permission denied: 'Data/Reno-Sushi.json'


## Open the Final JSON File with Pandas

In [27]:
df = pd.read_json(JSON_FILE)
df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,pms70Lg_z7c2y5ojukohBQ,miyabi-japanese-cuisine-reno,MIYABI Japanese Cuisine,https://s3-media1.fl.yelpcdn.com/bphoto/hE5VjD...,False,https://www.yelp.com/biz/miyabi-japanese-cuisi...,144,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 39.4035814750889, 'longitude': -1...",[],$$,"{'address1': '13967 S Virginia St', 'address2'...",+17754093520,(775) 409-3520,18553.049571
1,W9y4JkMqy2dFNwYc3Y5cKQ,hinoki-sushi-reno,Hinoki Sushi,https://s3-media3.fl.yelpcdn.com/bphoto/vwh7Ea...,False,https://www.yelp.com/biz/hinoki-sushi-reno?adj...,843,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.0,"{'latitude': 39.473410597855896, 'longitude': ...",[delivery],$$,"{'address1': '5270 Longley Ln', 'address2': ''...",+17754094994,(775) 409-4994,11412.584176
2,3FPi2yKCIh0Hh3iBg8faYA,tokyo-sushi-reno-2,Tokyo Sushi,https://s3-media4.fl.yelpcdn.com/bphoto/WVhsaZ...,False,https://www.yelp.com/biz/tokyo-sushi-reno-2?ad...,978,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.0,"{'latitude': 39.50491, 'longitude': -119.80335}",[],$$,"{'address1': '1999 S Virginia St', 'address2':...",+17758258828,(775) 825-8828,6560.327423
3,gExn3d0VMMJWlSICsGHGfg,ohana-sushi-sparks-3,Ohana Sushi,https://s3-media2.fl.yelpcdn.com/bphoto/nMfIDn...,False,https://www.yelp.com/biz/ohana-sushi-sparks-3?...,495,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 39.52066, 'longitude': -119.740751}",[],$$,"{'address1': '1560 S Stanford Way', 'address2'...",+17754539722,(775) 453-9722,10134.379359
4,mc-U-3T6UMN9x7SVm9Yn3A,hinoki-o-reno-2,Hinoki O,https://s3-media4.fl.yelpcdn.com/bphoto/ZsNOKx...,False,https://www.yelp.com/biz/hinoki-o-reno-2?adjus...,26,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",5.0,"{'latitude': 39.46823189149659, 'longitude': -...",[],NaN,"{'address1': '7500 Rancharrah Pkwy', 'address2...",+17754995818,(775) 499-5818,10421.916314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,4P2BALQPrfT-kDHhjsAUMw,7-eleven-south-lake-tahoe-2,7-Eleven,https://s3-media2.fl.yelpcdn.com/bphoto/fT1vwM...,False,https://www.yelp.com/biz/7-eleven-south-lake-t...,16,"[{'alias': 'convenience', 'title': 'Convenienc...",2.5,"{'latitude': 38.917536, 'longitude': -120.011056}",[pickup],$,"{'address1': '800 Emerald Bay Rd', 'address2':...",+15305410302,(530) 541-0302,71629.959385
256,xEG1a2-jC7QE4joTibANpQ,raleys-reno-2,Raley's,https://s3-media4.fl.yelpcdn.com/bphoto/TqyTau...,False,https://www.yelp.com/biz/raleys-reno-2?adjust_...,75,"[{'alias': 'grocery', 'title': 'Grocery'}, {'a...",3.5,"{'latitude': 39.53139031542699, 'longitude': -...",[],$$,"{'address1': '701 Keystone Ave', 'address2': '...",+17757475267,(775) 747-5267,2685.463468
257,NLRB3wjq_tY7WGL8HpPErw,raleys-o-n-e-market-reno,Raley's O-N-E Market,https://s3-media2.fl.yelpcdn.com/bphoto/u78Hud...,False,https://www.yelp.com/biz/raleys-o-n-e-market-r...,176,"[{'alias': 'grocery', 'title': 'Grocery'}, {'a...",2.5,"{'latitude': 39.39505095, 'longitude': -119.76...",[],$$,"{'address1': '18144 Wedge Pkwy', 'address2': '...",+17758508900,(775) 850-8900,18769.297468
258,BG_azyCQg7WzEpzZPsn15A,raleys-sparks-2,Raley's,https://s3-media2.fl.yelpcdn.com/bphoto/kolJCQ...,False,https://www.yelp.com/biz/raleys-sparks-2?adjus...,48,"[{'alias': 'grocery', 'title': 'Grocery'}, {'a...",4.0,"{'latitude': 39.55529322, 'longitude': -119.75...",[],$$,"{'address1': '2896 N Mccarran Blvd', 'address2...",+17753538800,(775) 353-8800,8461.634308


In [28]:
## convert the filename to a .csv.gz
csv_file = JSON_FILE.replace('.json','.csv.gz')
csv_file

'Data/Reno-Sushi.csv.gz'

In [29]:
## Save it as a compressed csv (to save space)
df.to_csv(csv_file, compression='gzip', index=False)

## Bonus: compare filesize with os module's `os.path.getsize`

In [30]:
size_json = os.path.getsize(JSON_FILE)
size_csv_gz = os.path.getsize(JSON_FILE.replace('.json','.csv.gz'))

print(f'JSON FILE: {size_json:,} Bytes')
print(f'CSV.GZ FILE: {size_csv_gz:,} Bytes')

print(f'the csv.gz is {size_json/size_csv_gz} times smaller!')

JSON FILE: 254,473 Bytes
CSV.GZ FILE: 37,889 Bytes
the csv.gz is 6.716276491857795 times smaller!


## Next Class: Processing the Results and Mapping 